## **SABR Model Calibration**

In [2]:
# -*- coding: utf-8 -*-
"""
Created on May 14 12:13:00 2024

@author: Bradley

SABR Model Calibration Example
"""

'\nCreated on May 14 12:13:00 2024\n\n@author: Bradley\n\nSABR Model Calibration Example\n'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime
import os

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')  

from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important; }</style>"))

from IPython.display import display
pd.set_option('expand_frame_repr', True) 
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180) 
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6) 
%config InlineBackend.figure_format = 'svg'
%matplotlib inline


### **SABR Model**

Recall the SABR model is given by
\begin{align*}
dF(t) &= \sigma(t)(F(t)+\theta)^{\beta}dW_1(t), F(0)=f\\
d\sigma(t) &= \nu\sigma(t)dW_2(t), \sigma(0)=\sigma_0\\
\end{align*}

where $W_1(t)$ and $W_2(t)$ are two correlated Wiener processes with correlation $\rho$, namely, $dW_1(t)dW_2(t) = \rho dt$.

- F: forward rate
- $\sigma$: volatility of forward rate
- $\nu$: volatility of volatility (volvol)
- $\theta$: shift parameter to avoid negative rates


#### **SABR Implied Volatility**

In [4]:
def sabr_normal_vol(f, K, t, sigma0, beta, rho, volvol):
    """
    Returns: Normal implied volatility as per the SABR model (without shift)
    """
    
    # ATM case, f = K, f_mid = f
    if abs(f-K) < 1e-4:  
        A = -beta * (2-beta) * sigma0**2 / (24 * f**(2-2*beta))
        B = rho * sigma0 * volvol * beta / (4 * f**(1-beta))
        C = (2-3*rho**2) * volvol**2 / 24
        sigma_N = sigma0 * f**beta * (1+(A+B+C)*t)

    # General case, f != K
    # Following the original paper, f_mid = sqrt(f * K)
    else:  
        if beta != 1:
            zeta_K = (volvol/(sigma0*(1-beta))) * (f**(1-beta) - K**(1-beta))
        else:
            zeta_K = volvol * np.log(f/K) / sigma0
        x_zeta = (1 / volvol) * np.log((np.sqrt(1 - 2 * rho * zeta_K + zeta_K**2) + zeta_K - rho) / (1-rho))        
        A = (1/24) * (beta**2-2*beta) * sigma0**2 * f**(beta-1) * K**(beta-1)
        B = (1/4) * rho * volvol * sigma0 * f**((beta-1)/2) * K**((beta-1)/2)
        C = (1/24) * (2-3*rho**2) * volvol**2
        sigma_N = (f-K) / x_zeta * (1 + (A+B+C)*t)

    return sigma_N

def sabr_lognormal_vol(f, K, t, sigma0, beta, rho, volvol):
    """
    Returns: Lognormal implied volatility as per the SABR model (without shift)
    """

    # ATM case
    if abs(f-K) < 1e-4:  
        A = (beta-1)**2 * sigma0**2 / (24 * f**(2-2*beta))
        B = rho * sigma0 * volvol * beta / (4 * f**(1-beta))
        C = (2-3*rho**2) * volvol**2 / 24
        sigma_LN = sigma0 * f**(beta-1) * (1+(A+B+C)*t)

    # General case
    else:
        if beta != 1:
            zeta_K = (volvol/(sigma0*(1-beta))) * (f**(1-beta) - K**(1-beta))
        else:
            zeta_K = volvol * np.log(f/K) / sigma0
        x_zeta = (1 / volvol) * np.log((np.sqrt(1 - 2 * rho * zeta_K + zeta_K**2) + zeta_K - rho) / (1-rho))       
        A = (1/24) * (beta-1)**2 * sigma0**2 * f**(beta-1) * K**(beta-1)
        B = (1/4) * rho * volvol * sigma0 * f**((beta-1)/2) * K**((beta-1)/2)
        C = (1/24) * (2-3*rho**2) * volvol**2
        sigma_LN = np.log(f/K) / x_zeta * (1 + (A+B+C)*t)
        
    return sigma_LN


In [5]:
# Example usage
f = 100
K = 50
T = 1
volvol = 0.2
sigma0 = 0.3
beta = 0.5
rho = -0.1

volatility = sabr_normal_vol(f, K, T, sigma0, beta, rho, volvol)
print(f"SABR Implied Normal Volatility: {volatility:.4f}")

log_volatility = sabr_lognormal_vol(f, K, T, sigma0, beta, rho, volvol)
print(f"SABR Implied Lognormal Volatility: {log_volatility:.4f}")

SABR Implied Normal Volatility: 5.0140
SABR Implied Lognormal Volatility: 0.0695


Let's verify the results with built-in package that calculates the implied volatility of the SABR model.

In [6]:
from pysabr import Hagan2002LognormalSABR, Hagan2002NormalSABR

def v_atm_n(f, t, sigma0, beta, rho, volvol):
    f_av = f
    A = - beta * (2 - beta) * sigma0**2 / (24 * f_av**(2 - 2 * beta))
    B = rho * sigma0 * volvol * beta / (4 * f_av**(1 - beta))
    C = (2 - 3 * rho**2) * volvol**2 / 24
    v_atm_n = sigma0 * f**beta * (1 + (A + B + C) * t)
    return v_atm_n

f = 100
K = 50
T = 1
volvol = 0.2
sigma0 = 0.3
beta = 0.5
rho = -0.1

# calculate ATM volatility since the built-in method needs atm vol as the input rather than sigma0
atm_vol = v_atm_n(f, T, sigma0, beta, rho, volvol)
# print(f"ATM Volatility: {atm_vol:.4f}")

sabr = Hagan2002NormalSABR(f=f, shift=0, t=T, v_atm_n=atm_vol, beta=beta, rho=rho, volvol=volvol)
volatility = sabr.normal_vol(k=K) 
print(f"SABR Implied Normal Volatility (built-in): {volatility:.4f}")

sabr = Hagan2002LognormalSABR(f=f, shift=0, t=T, v_atm_n=atm_vol, beta=beta, rho=rho, volvol=volvol)
log_volatility = sabr.lognormal_vol(k=K) 
print(f"SABR Implied Lognormal Volatility (built-in): {log_volatility:.4f}")


SABR Implied Normal Volatility (built-in): 5.0542
SABR Implied Lognormal Volatility (built-in): 0.0693


The SABR implied normal and lognormal volatilities we calculate are consistent with the results obtained in built-in functions.

### **Model Parameter Calibration**

In [7]:
# Implied volatilities for different strikes at various maturities
# unit: %
# K: bps from ATM
data = {
    "K": [-250, -200, -150, -100, -50, -25, 0, 25, 50, 100, 150, 200, 250],
    "0.25": [48.69, 41.70, 37.59, 35.35, 34.43, 34.31, 34.36, 34.52, 34.77, 35.41, 36.16, 36.92, 37.67],
    "0.5": [45.40, 39.75, 36.29, 34.25, 33.20, 32.95, 32.82, 32.80, 32.86, 33.14, 33.54, 34.00, 34.49],
    "1": [41.10, 36.71, 33.83, 31.99, 30.90, 30.57, 30.35, 30.22, 30.16, 30.21, 30.41, 30.68, 31.01],
    "2": [36.31, 33.05, 30.74, 29.14, 28.11, 27.77, 27.52, 27.35, 27.24, 27.19, 27.29, 27.48, 27.73],
    "3": [33.04, 30.26, 28.18, 26.65, 25.60, 25.22, 24.93, 24.72, 24.58, 24.45, 24.48, 24.63, 24.84],
    "5": [29.69, 27.33, 25.45, 23.99, 22.90, 22.49, 22.15, 21.89, 21.70, 21.47, 21.42, 21.49, 21.64]
}
df = pd.DataFrame(data)


# Forward rates for each maturity
# unit: %
forward_rates = {
    "0.25": 3.510,
    "0.5": 3.632,
    "1": 3.872,
    "2": 4.313,
    "3": 4.657,
    "5": 5.030
}
df


,K,0.25,0.5,1,2,3,5
0,-250,48.69,45.40,41.10,36.31,33.04,29.69
1,-200,41.70,39.75,36.71,33.05,30.26,27.33
2,-150,37.59,36.29,33.83,30.74,28.18,25.45
3,-100,35.35,34.25,31.99,29.14,26.65,23.99
4,-50,34.43,33.20,30.90,28.11,25.60,22.90
5,-25,34.31,32.95,30.57,27.77,25.22,22.49
6,0,34.36,32.82,30.35,27.52,24.93,22.15
7,25,34.52,32.80,30.22,27.35,24.72,21.89
8,50,34.77,32.86,30.16,27.24,24.58,21.70
9,100,35.41,33.14,30.21,27.19,24.45,21.47


**SABR Normal Model using NLS Optimization**

In [8]:
# Normal model

from scipy.optimize import curve_fit

# For ease of optimization, we vectorize the SABR normal volatility function regarding strike K and change parameter order
def sabr_normal_vol(K, sigma0, rho, volvol, f, t, beta=0.5):
    """
    Returns: Normal implied volatility as per the SABR model (without shift)
    """
    # Vectorized condition to check if f is approximately equal to K
    close_to_atm = np.abs(f - K) < 0.01
    # ATM case calculations (f ~ K)
    A_atm = -beta * (2-beta) * sigma0**2 / (24 * f**(2-2*beta))
    B_atm = rho * sigma0 * volvol * beta / (4 * f**(1-beta))
    C_atm = (2-3*rho**2) * volvol**2 / 24
    sigma_N_atm = sigma0 * f**beta * (1 + (A_atm + B_atm + C_atm) * t)

    # Non-ATM case calculations (f != K)
    if beta != 1:
        zeta_K = (volvol/(sigma0*(1-beta))) * (f**(1-beta) - K**(1-beta))
    else:
        zeta_K = volvol * np.log(f/K) / sigma0
    x_zeta = (1 / volvol) * np.log((np.sqrt(1 - 2 * rho * zeta_K + zeta_K**2) + zeta_K - rho) / (1-rho))        
    A = (1/24) * (beta**2-2*beta) * sigma0**2 * f**(beta-1) * K**(beta-1)
    B = (1/4) * rho * volvol * sigma0 * f**((beta-1)/2) * K**((beta-1)/2)
    C = (1/24) * (2-3*rho**2) * volvol**2
    sigma_N = (f-K) / x_zeta * (1 + (A + B + C) * t)

    # Use np.where to combine ATM and non-ATM cases
    sigma_N = np.where(close_to_atm, sigma_N_atm, sigma_N)

    # NOTE: We multiply normal vol by 10000 to convert the output to percentage
    return sigma_N * 10000

maturity = [0.25, 0.5, 1, 2, 3, 5]
param_N = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_N.index.name = 'Maturity'

for mat in maturity:
    initial_guess = [0.1, 0.0, 0.3]  # sigma0, rho, volvol
    forward_rate = forward_rates[str(mat)] * 0.01
    
    K_array = df['K'].to_numpy() * 0.0001 + forward_rate
    vol_array = df[str(mat)].to_numpy() 
    params, covariance = curve_fit(lambda K, sigma0, rho, volvol: sabr_normal_vol(K, sigma0, rho, volvol, forward_rate, mat), 
                                   xdata=K_array, ydata=vol_array, p0=initial_guess, maxfev=5000)
    param_N.loc[mat] = params

param_N

,sigma0,rho,volvol
Maturity,,,
0.25,0.018293,-0.457002,0.234944
0.50,0.017222,-0.499198,0.208936
1.00,0.015449,-0.520543,0.180637
2.00,0.013282,-0.516947,0.150742
3.00,0.011590,-0.525420,0.134966
5.00,0.009919,-0.547108,0.120016


**SABR Normal Model using pysabr Package**

In [9]:
# Normal model
from pysabr import Hagan2002NormalSABR

maturity = [0.25, 0.5, 1, 2, 3, 5]

param_N = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_N.index.name = 'Maturity'


# fit SABR model for each maturity
for mat in maturity:
    forward_rate = forward_rates[str(mat)] * 0.01
    sabr = Hagan2002NormalSABR(f=forward_rate, shift=0, t=mat, beta=0.5) # fix beta=0.5
    K_array = df['K'].to_numpy() * 0.0001 + forward_rate
    
    # does not multiply by 0.01 for vol array since in the package implementation it is multiplied when calculating squared error
    vol_array = df[str(mat)].to_numpy() 
    [sigma0, rho, volvol] = sabr.fit(K_array, vol_array)
    param_N.loc[mat] = [sigma0, rho, volvol]
    
param_N

,sigma0,rho,volvol
Maturity,,,
0.25,0.018226,-0.441186,0.229508
0.50,0.017178,-0.487844,0.203775
1.00,0.015417,-0.512485,0.176431
2.00,0.013257,-0.511163,0.147836
3.00,0.011568,-0.520563,0.132636
5.00,0.009901,-0.543454,0.117981


**SABR Log-normal Model using NLS Optimization**

In [10]:
# Log-normal model

from scipy.optimize import curve_fit

# For ease of optimization, we vectorize the SABR lognormal volatility function regarding strike K and change parameter order
def sabr_lognormal_vol(K, sigma0, rho, volvol, f, t, beta=0.5):
    """
    Returns: Lognormal implied volatility as per the SABR model (without shift)
    """
    # Vectorized condition to check if f is approximately equal to K
    close_to_atm = np.abs(f - K) < 1e-4 
    
    # ATM case calculations
    A_atm = (beta-1)**2 * sigma0**2 / (24 * f**(2-2*beta))
    B_atm = rho * sigma0 * volvol * beta / (4 * f**(1-beta))
    C_atm = (2-3*rho**2) * volvol**2 / 24
    sigma_LN_atm = sigma0 * f**(beta-1) * (1 + (A_atm + B_atm + C_atm) * t)

    # General case calculations
    if beta != 1:
        zeta_K = (volvol / (sigma0 * (1-beta))) * (f**(1-beta) - K**(1-beta))
    else:
        zeta_K = volvol * np.log(f/K) / sigma0
    x_zeta = (1 / volvol) * np.log((np.sqrt(1 - 2 * rho * zeta_K + zeta_K**2) + zeta_K - rho) / (1-rho))
    A = (1/24) * (beta-1)**2 * sigma0**2 * f**(beta-1) * K**(beta-1)
    B = (1/4) * rho * volvol * sigma0 * f**((beta-1)/2) * K**((beta-1)/2)
    C = (1/24) * (2-3*rho**2) * volvol**2
    sigma_LN = np.log(f/K) / x_zeta * (1 + (A + B + C) * t)

    # Combine ATM and non-ATM cases using np.where
    sigma_LN = np.where(close_to_atm, sigma_LN_atm, sigma_LN)

    # NOTE: Need to multiply log-normal vol by 100 to compare with the implied volatility
    return sigma_LN * 100


maturity = [0.25, 0.5, 1, 2, 3, 5]
param_LN = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_LN.index.name = 'Maturity'

for mat in maturity:
    initial_guess = [0.1, 0.0, 0.3]  # sigma0, rho, volvol
    forward_rate = forward_rates[str(mat)] * 0.01
    
    K_array = df['K'].to_numpy() * 0.0001 + forward_rate
    vol_array = df[str(mat)].to_numpy() 
    params, covariance = curve_fit(lambda K, sigma0, rho, volvol: sabr_lognormal_vol(K, sigma0, rho, volvol, forward_rate, mat), 
                                   xdata=K_array, ydata=vol_array, p0=initial_guess, maxfev=5000)
    param_LN.loc[mat] = params

param_LN


,sigma0,rho,volvol
Maturity,,,
0.25,0.063806,0.373809,0.540948
0.50,0.061747,0.313424,0.458956
1.00,0.058561,0.243499,0.405545
2.00,0.055353,0.173489,0.382649
3.00,0.051675,0.092225,0.373275
5.00,0.047173,-0.010828,0.356029


**SABR Log-normal Model using pysabr Package**

In [11]:
# Log-normal model
from pysabr import Hagan2002LognormalSABR

maturity = [0.25, 0.5, 1, 2, 3, 5]

param_LN = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_LN.index.name = 'Maturity'


# fit SABR model for each maturity
for mat in maturity:
    forward_rate = forward_rates[str(mat)] * 0.01
    sabr = Hagan2002LognormalSABR(f=forward_rate, shift=0, t=mat, beta=0.5) # fix beta=0.5
    K_array = df['K'].to_numpy() * 0.0001 + forward_rate
    
    # does not multiply by 0.01 for vol array since in the package implementation it is multiplied when calculating squared error
    vol_array = df[str(mat)].to_numpy() 
    [sigma0, rho, volvol] = sabr.fit(K_array, vol_array)
    param_LN.loc[mat] = [sigma0, rho, volvol]
    
param_LN

,sigma0,rho,volvol
Maturity,,,
0.25,0.063881,0.370195,0.547521
0.50,0.061863,0.309781,0.464934
1.00,0.058705,0.240046,0.411120
2.00,0.055509,0.170550,0.387989
3.00,0.051762,0.090486,0.377172
5.00,0.047140,-0.011172,0.356810


Optimization results are consistent with the built-in package results.

The result is also consistent with the empirical properties:
- The vol in vol parameter shows a persistent stable term structure and is monotonically decreasing with maturity
- The typical range for volvol is between 0.2 and 2